In [1]:
import random
def choose_random_item(lst):
    return random.choice(lst)

def random_int(fro, to):
    return random.randint(fro, to)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand
import random
import datetime
import json
def choose_random_item(lst):
    return random.choice(lst)

def random_int(fro, to):
    return random.randint(fro, to)
# Create a SparkSession
spark = SparkSession.builder.appName("MonsterTracker").config("spark.jars","work/data/commons-pool2-2.11.1.jar,work/data/spark-sql-kafka-0-10_2.12-3.4.0.jar,work/data/spark-streaming-kafka-0-10-assembly_2.12-3.4.0.jar").getOrCreate()

# Read the file into a DataFrame
df = spark.read.csv("./work/data/dnd_monsters.csv",  header=True).where("str is not null").select("name", "str")

# Randomly select 5 lines
random_lines = df.orderBy(rand()).limit(5)

# Show the selected lines
random_lines.show(truncate=False)
list_tuples = random_lines.rdd.map(tuple).collect()


for monster in range(1,200):
    random_tuple = random.choice(list_tuples)
    data = {
        "name": random_tuple[0],
        "str": random_tuple[0],
        "ts": str(datetime.datetime.now()),
        "lat": random_int(-90, 90),
        "long": random_int(-180, 180)
    }

    # Serialize the dictionary to a JSON string
    json_string = json.dumps(data)

    # Print the JSON string
    print(json_string)

In [2]:
pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 8.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from geopy.geocoders import Nominatim

# Create a geocoder instance
geolocator = Nominatim(user_agent="my_geocoder")

# Define the latitude and longitude coordinates
latitude = 51.5074
longitude = -0.1278

# Reverse geocode to get the location information
location = geolocator.reverse((latitude, longitude), language='en')

# Extract the country from the location information, with a random lat and long it might not have a country
country = "Sea"
try:
    country = location.raw['address']['country']
except:
    print("out to sea")
print(country)

# Print the country
print(country)

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand
import random
import datetime
import json

from geopy.geocoders import Nominatim

# Create a geocoder instance
geolocator = Nominatim(user_agent="my_geocoder")


def choose_random_item(lst):
    return random.choice(lst)

def random_int(fro, to):
    return random.randint(fro, to)
# Create a SparkSession
spark = SparkSession.builder.appName("MonsterTracker").config("spark.jars","work/data/commons-pool2-2.11.1.jar,work/data/spark-sql-kafka-0-10_2.12-3.4.0.jar,work/data/spark-streaming-kafka-0-10-assembly_2.12-3.4.0.jar").getOrCreate()


# Read the file into a DataFrame
df = spark.read.csv("./work/data/dnd_monsters.csv",  header=True).where("str is not null").select("name", "str")

# Randomly select 5 lines
random_lines = df.orderBy(rand()).limit(5)

# Show the selected lines
random_lines.show(truncate=False)
list_tuples = random_lines.rdd.map(tuple).collect()


for monster in range(1,200):
    random_tuple = random.choice(list_tuples)
    lat = random_int(-90, 90)
    long = random_int(-180, 180)

    # Reverse geocode to get the location information
    location = geolocator.reverse((lat, long), language='en')

    # Extract the country from the location information
    country = "Sea"
    try:
        country = location.raw['address']['country']
    except:
        print("out to sea")
    print(country)
    data = {
    "name": random_tuple[0],
    "str": random_tuple[1],
    "ts": str(datetime.datetime.now()),
    "lat": lat,
    "long": long,
    "country": country
    }

    json_string = json.dumps(data)

    # Print the JSON string
    print(json_string)
    data = [(json_string,),]
    df = spark.createDataFrame(data, ["value"])
    df.show(1, False)
    # # Write the DataFrame to Kafka
    df.write \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:9092") \
        .option("topic", "monster_movements") \
        .save()

+-----------------+----+
|name             |str |
+-----------------+----+
|octopus          |4.0 |
|kobold-inventor  |7.0 |
|black-pudding    |16.0|
|peryton          |16.0|
|yuan-ti-pureblood|11  |
+-----------------+----+

out to sea
Sea
{"name": "octopus", "str": "4.0", "ts": "2023-08-01 09:46:31.254206", "lat": -38, "long": -153, "country": "Sea"}
+-----------------------------------------------------------------------------------------------------------------+
|value                                                                                                            |
+-----------------------------------------------------------------------------------------------------------------+
|{"name": "octopus", "str": "4.0", "ts": "2023-08-01 09:46:31.254206", "lat": -38, "long": -153, "country": "Sea"}|
+-----------------------------------------------------------------------------------------------------------------+

out to sea
Sea
{"name": "peryton", "str": "16.0", "ts": "2023-08

In [5]:
# Creating a SparkSession with the extra jars we need for Kafka
stream_spark = SparkSession.builder.appName("Monster_Stream").config("spark.jars","work/data/commons-pool2-2.11.1.jar,work/data/spark-sql-kafka-0-10_2.12-3.4.0.jar,work/data/spark-streaming-kafka-0-10-assembly_2.12-3.4.0.jar").getOrCreate()

# Read Kafka as a stream, you can read it as a batch also.  Simple read everything
# on the topic into one DataFrame going back to the beginning
stream_spark_df = stream_spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9092").option("subscribe", "monster_movement").option("startingOffsets","earliest").load()

# Commented out code which would make the output more readable
# df = df.selectExpr("CAST(key AS STRING) as Key", "CAST(value AS STRING) as Value")

stream_spark_df = stream_spark_df.selectExpr("CAST(value AS STRING) as Value")
# Here we take the previous stream from Kafka and write that stream to the console 
rawQuery = stream_spark_df.writeStream.format("console").outputMode("append").start()


# Once you start the stream you either end it or keep it listening for more data with
# the awaitTermination option
rawQuery.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 